<a href="https://colab.research.google.com/github/ary4m4n03/Golden-Harvesting-A-Predictive-Model-for-Apple-Quality-Assurance/blob/main/Golden_Harvesting_A_Predictive_Model_for_Apple_Quality_Assurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import and Read Dataset

In [4]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("nelgiriyewithana/apple-quality")

print("Path to dataset files:", path)

data=pd.read_csv("/kaggle/input/apple-quality/apple_quality.csv")

data.head()

Path to dataset files: /kaggle/input/apple-quality


,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,good
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,good


Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

Data Preprocessing

Exploratory Data Analysis

Model Building

Performance Testing and Hyperparameter Tuning

Saving Best Model